#  Подход 1: градиентный бустинг "в лоб"

In [1]:
import json
import bz2
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import time
import datetime

In [2]:
kf = KFold(n_splits=5, shuffle=True, random_state=241)

In [3]:
df = pd.read_csv('./features.csv', index_col='match_id')

In [4]:
X = df.copy(deep=True)
y = df['radiant_win'].copy(deep=True)

In [5]:
col = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
       'barracks_status_radiant', 'barracks_status_dire']

In [6]:
for i in col:
    del X[i]
len(X.columns)

102

In [7]:
# Находим признаки имеющие пропуски
a = X.count(axis=0)
print(a[a <97230])

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


In [8]:
X = X.fillna(0).copy(deep=True)  # Приравниваем пропущенные значения к 0

In [9]:
a = X.count(axis=0)  # Проверяем все ли значения указаны
print(max(a))
print(min(a))
print(a[a <97230])

97230
97230
Series([], dtype: int64)


In [10]:
def scor(X, y, n, md):
    gbc = GBC(n_estimators=n, verbose=False, random_state=241, max_depth=md)
    scor = cross_val_score(gbc, X, y, scoring = 'roc_auc', cv = kf)
    return scor

In [11]:
  #  Кросс-валидация для градиентного бустинга с 10, 20, 30 деревьями
list1 = []
all_scores = {}
time = {}
for i in [10, 20, 30]:
    start_time = datetime.datetime.now()
    scores = scor(X, y, i, 3)
    r = scores.mean()
    all_scores['n_estimators=' + str(i)] = [scores]
    list1 += [('n_estimators=' + str(i) + ' ' + str(r))]
    time[('n_estimators=' + str(i) + ' Time elapsed:')] = datetime.datetime.now() - start_time

In [12]:
all_scores

{'n_estimators=10': [array([0.66943496, 0.65627754, 0.66390454, 0.66281223, 0.66950933])],
 'n_estimators=20': [array([0.68939011, 0.67567277, 0.68201026, 0.67977584, 0.68741889])],
 'n_estimators=30': [array([0.69641668, 0.68365441, 0.68731862, 0.68728458, 0.69280674])]}

In [13]:
list1

['n_estimators=10 0.6643877206345741',
 'n_estimators=20 0.6828535735340823',
 'n_estimators=30 0.6894962060591201']

In [14]:
time

{'n_estimators=10 Time elapsed:': datetime.timedelta(seconds=64, microseconds=98803),
 'n_estimators=20 Time elapsed:': datetime.timedelta(seconds=109, microseconds=534147),
 'n_estimators=30 Time elapsed:': datetime.timedelta(seconds=155, microseconds=939137)}

In [15]:
#  Кросс-валидация для градиентного бустинга 50 деревьев, глубина 2
start_time = datetime.datetime.now()
scores = scor(X, y, 30, 3)
print(datetime.datetime.now() - start_time)
r = scores.mean()
print(r)

0:02:36.118256
0.6894962060591201


Кросс-валидация для градиентного бустинга с 30 деревьями проводилась 171 секунду, 
качество при этом ровняется scores.mean = 0.6313. Я попробовал использовать 50 деревьев 
среднее качество по кросс-валидации(5 блоков) получилось 0.6345,  время (seconds=146). 
При моих данных получается , что имеет смысл использовать больше 30 деревьев. 
Чтобы ускорить его обучение при увеличении количества деревьев, я предложил  использовать 
для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество.

#  Подход 2: логистическая регрессия

In [ ]:
import json
import bz2
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
df = pd.read_csv('./features.csv', index_col='match_id')

In [ ]:
y = df['radiant_win'].copy(deep=True)
X = df.copy(deep=True).fillna(0)  # Приравниваем пропущенные значения к 0
col = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire',
       'barracks_status_radiant', 'barracks_status_dire']
for i in col:
    del X[i]     #  Удаляем признаки, связанные с итогами матча

In [ ]:
# Масштабируем признаки
def scaler_(X):
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaler = scaler.transform(X)
    return X_scaler

In [ ]:
# Подбираем параметр С
def grid_(X, y):
    X = scaler_(X)
    start_time = datetime.datetime.now()
    grid = {'C': np.power(10.0, np.arange(-5, 6))}
    cv = KFold(n_splits=5, shuffle=True, random_state=241)
    clf = LogisticRegression(random_state=241, solver='lbfgs', class_weight='balanced', warm_start=True)
    gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv=cv)
    gs.fit(X, y)
    time = datetime.datetime.now() - start_time
    global C
    C = gs.best_params_['C']
    print(gs.best_score_)
    print('C =', gs.best_params_['C'])
    print('time:', time)

In [ ]:
# Оценивает качество логистической регрессии с помощью кросс-валидации(5 блоков) с найденным параметром С
def log_reg(X, y):
    global clf
    X = scaler_(X)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(random_state=241, solver='lbfgs', class_weight='balanced', warm_start=True, C=0.001)
    kf = KFold(n_splits=5, shuffle=True, random_state=241)
    scores = cross_val_score(clf, X, y, scoring = 'roc_auc', cv = kf)
    time = (datetime.datetime.now() - start_time)
    r = scores.mean()
    print(scores)
    print(r)
    print('time:', time)

###  Качество логистической регрессии над всеми исходными признакам

In [ ]:
grid_(X, y)

In [ ]:
log_reg(X, y)

###  Качество логистической регрессии, при удаление категориальных признаков 

In [ ]:
col_2 = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero',  'd5_hero']

In [ ]:
#  удаление категориальных признаков 
for i in col_2:
    del X[i]
len(X.columns)

In [ ]:
grid_(X, y)  #  Подбираем С на данных без категориальных признаков

In [ ]:
log_reg(X, y)  #  качество логистической регрессии на данных без категориальных признаков

### Включаем в работу исключенные ранее признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. 

In [ ]:
X_3 = df[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero',  'd5_hero']].copy(deep=True)

In [ ]:
unique_hero = np.unique(X_3, axis=0).reshape((1,972300))[0]

In [ ]:
unique_hero = np.unique(unique_hero, axis=0)

In [ ]:
len(unique_hero)

In [ ]:
N = len(unique_hero)

In [ ]:
X_pick = np.zeros((df.shape[0], 112))

for i, match_id in enumerate(df.index):
    for p in range(5):
        X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [ ]:
X_pick = pd.DataFrame(X_pick)

In [ ]:
X_pick = X_pick.fillna(0)

In [ ]:
X = np.hstack((X, X_pick))

In [ ]:
grid_(X, y)

In [ ]:
log_reg(X, y)

### Проверка финальной модели

In [ ]:
df_1 = pd.read_csv('./features_test.csv', index_col='match_id')

In [ ]:
X_1 = df_1.copy(deep=True).fillna(0)  # Приравниваем пропущенные значения к 0
    
#  преобразование категориальных признаков 
for i in col_2:
    del X_1[i]

X_pick = np.zeros((df_1.shape[0], 112))

for i, match_id in enumerate(df_1.index):
    for p in range(5):
        X_pick[i, df_1.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df_1.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_pick = pd.DataFrame(X_pick)
X_pick = X_pick.fillna(0)
X_1 = np.hstack((X_1, X_pick))

In [ ]:
X = scaler_(X)
clf = LogisticRegression(random_state=241, solver='lbfgs', class_weight='balanced', warm_start=True, C=0.001)
clf.fit(X,y)


In [ ]:
X_1 = scaler_(X_1)
pred = clf.predict_proba(X_1)[:, 1]

In [ ]:
X_4 = df_1['start_time'].copy(deep=True)

In [ ]:
X_4 = pd.DataFrame(X_4)

In [ ]:
X_4 = X_4.rename(columns={'start_time': 'radiant_win'})

In [ ]:
X_4['radiant_win'] = pred

In [ ]:
#  минимальное и максимальное значение прогноза на тестовой выборке лучшего из алгоритмов
print('min =', min(X_4['radiant_win'])) 
print('max =', max(X_4['radiant_win']))

In [ ]:
X_4.to_csv('submission_1.csv')